In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy.special import logit, expit
from six.moves import cPickle

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [3]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2),
    max_features=20000)
word_vectorizer.fit(all_text)
print('A')
train_word_features = word_vectorizer.transform(train_text)
print('B')
test_word_features = word_vectorizer.transform(test_text)
with open("word_vectorizer.pkl", 'wb') as f:
    cPickle.dump(word_vectorizer, f)

A
B


In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=10000)
char_vectorizer.fit(all_text)
print('A')
train_char_features = char_vectorizer.transform(train_text)
print('B')
test_char_features = char_vectorizer.transform(test_text)

In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [ ]:
losses = []
predictions = {'id': test['id']}
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')
    cv_loss = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))
    classifier.fit(train_features, train_target)
    with open(class_name + "_model.pkl", 'wb') as f:
        cPickle.dump(classifier, f)
    print(class_name + " completed")
    # predictions[class_name] = expit(logit(classifier.predict_proba(test_features)[:, 1]) - 0.5)